# Create vector-index

In [ ]:
# <tab: 1-Modality>
from superduperdb import VectorIndex, Listener

jobs, _ db.add(
    VectorIndex(
        'my-vector-index',
        indexing_listener=Listener(
            key='<my_key>',      # the `Document` key `model` should ingest to create embedding
            select=select,       # a `Select` query telling which data to search over
            model=model,         # a `_Predictor` how to convert data to embeddings
        )
    )
)

In [ ]:
# <tab: 2-Modalities>
from superduperdb import VectorIndex, Listener

jobs, _ db.add(
    VectorIndex(
        'my-vector-index',
        indexing_listener=Listener(
            key='<my_key>',      # the `Document` key `model` should ingest to create embedding
            select=select,       # a `Select` query telling which data to search over
            model=model,         # a `_Predictor` how to convert data to embeddings
        ),
        compatible_listener=Listener(
            key='<other_key>',      # the `Document` key `model` should ingest to create embedding
            model=compatible_model,         # a `_Predictor` how to convert data to embeddings
            active=False,
        )
    )
)